In [2]:
import os

In [3]:
import praw
import openai

In [4]:
client_id = os.getenv("REDDIT_CLIENT")
client_secret = os.getenv("REDDIT_SECRET")

In [7]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent="ai sentiment example (by u/karidwyn)"
                    )

In [9]:
for submission in reddit.subreddit("stocks").hot(limit=5):
    print(submission.title)

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
r/Stocks Daily Discussion & Technicals Tuesday - Mar 28, 2023
Alibaba to split into 6 units
Apple (AAPL) to Spend Billions More on Entertainment
Disney will start laying off employees this week, CEO Bob Iger says in memo


In [10]:
subreddit_stocks = reddit.subreddit("stocks")

In [11]:
subreddit_stocks.title

'Stocks - Investing and trading for all'

In [12]:
subreddit_stocks.accounts_active

4881

In [18]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    #print the top 2 comments per title submission
    counter = 0
    for comment in submission.comments:
        if comment.body == '[deleted]':
            pass
        else:
            print(comment.body)
        counter+=1
        if counter == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
I am 26, so I have several decades until retirement. My goal is to hold the stocks in my portfolio perpetually, though I'm not reserved to make adjustments and sell when a) I see an opportunity to take profits b) my thesis changes. My long term goal is passive income generation, though I don't chase yields. I don't pay much attention to valuation or macro, as I think that since I plan on holding these stocks for decades while buying every month, those don't matter to me much. I invest regularly every month, so I'm okay with buying at a higher valuation at times and lower valuation at other times. In my opinion, buying at the best possible time is more difficult than buying a good company. That said, if I see an undervalued stock I like, I don't hesitate to buy more and when I make my regular contributions, I try weigh towards stocks that might have been beaten down or seem undervalued. My last review in one of these was [Dec 2021